<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
import sklearn.preprocessing as preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import itertools
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
#from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')


In [2]:
## To display  all the interactive output without using the print function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
df=pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
# lowercase column name
df.columns = map(str.lower, df.columns)
df.columns

Index(['customerid', 'gender', 'seniorcitizen', 'partner', 'dependents',
       'tenure', 'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod', 'monthlycharges', 'totalcharges', 'churn'],
      dtype='object')

In [5]:
# customerid not useful better to drop it
df.drop(['customerid'], axis=1, inplace=True)

In [6]:
df['totalcharges'].unique()
#Replacing spaces with null values in total charges column
df['totalcharges'] = df["totalcharges"].replace(" ",np.nan)
df.info()

array(['29.85', '1889.5', '108.15', ..., '346.45', '306.6', '6844.5'],
      dtype=object)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
gender              7043 non-null object
seniorcitizen       7043 non-null int64
partner             7043 non-null object
dependents          7043 non-null object
tenure              7043 non-null int64
phoneservice        7043 non-null object
multiplelines       7043 non-null object
internetservice     7043 non-null object
onlinesecurity      7043 non-null object
onlinebackup        7043 non-null object
deviceprotection    7043 non-null object
techsupport         7043 non-null object
streamingtv         7043 non-null object
streamingmovies     7043 non-null object
contract            7043 non-null object
paperlessbilling    7043 non-null object
paymentmethod       7043 non-null object
monthlycharges      7043 non-null float64
totalcharges        7032 non-null object
churn               7043 non-null object
dtypes: float64(1), int64(2), object(17)
memory usage: 1.1+ MB


In [7]:
print((df['totalcharges'].isna().sum()/len(df))*100) # count the percentage of missing data
#Dropping null values from total charges column which contain .15% missing data 
df.dropna(inplace=True)

0.1561834445548772


In [8]:
# checking missing data
df.isna().sum()

gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

In [9]:
df['onlinesecurity'].value_counts()
df['onlinebackup'].value_counts()
df['deviceprotection'].value_counts()
df['techsupport'].value_counts()
df['streamingtv'].value_counts()
df['streamingmovies'].value_counts()

No                     3497
Yes                    2015
No internet service    1520
Name: onlinesecurity, dtype: int64

No                     3087
Yes                    2425
No internet service    1520
Name: onlinebackup, dtype: int64

No                     3094
Yes                    2418
No internet service    1520
Name: deviceprotection, dtype: int64

No                     3472
Yes                    2040
No internet service    1520
Name: techsupport, dtype: int64

No                     2809
Yes                    2703
No internet service    1520
Name: streamingtv, dtype: int64

No                     2781
Yes                    2731
No internet service    1520
Name: streamingmovies, dtype: int64

In [10]:
#replace 'No internet service' to No for the following columns
replace_cols = [ 'onlinesecurity', 'onlinebackup', 'deviceprotection',
                'techsupport','streamingtv', 'streamingmovies']
for i in replace_cols : 
    df[i]  = df[i].replace({'No internet service' : 'No'})

In [11]:
df['onlinesecurity'].unique()
df['onlinesecurity'].value_counts()
df['onlinebackup'].value_counts()
df['deviceprotection'].value_counts()
df['techsupport'].value_counts()
df['streamingtv'].value_counts()
df['streamingmovies'].value_counts()

array(['No', 'Yes'], dtype=object)

No     5017
Yes    2015
Name: onlinesecurity, dtype: int64

No     4607
Yes    2425
Name: onlinebackup, dtype: int64

No     4614
Yes    2418
Name: deviceprotection, dtype: int64

No     4992
Yes    2040
Name: techsupport, dtype: int64

No     4329
Yes    2703
Name: streamingtv, dtype: int64

No     4301
Yes    2731
Name: streamingmovies, dtype: int64

In [12]:
# replacing values
df['seniorcitizen']=df['seniorcitizen'].replace({1:'Yes',0:"No"})
df['seniorcitizen'].value_counts()

No     5890
Yes    1142
Name: seniorcitizen, dtype: int64

In [13]:
df['churn'] = df['churn'].apply(lambda x: x.strip().replace("Yes", "1").replace("No", "0"))
df['churn'] = df['churn'].astype('int')

In [14]:
#### onehot encoding all caterogical data
df = pd.get_dummies(df, drop_first=True)

In [15]:
## Define X, y and split data into training and testing
from sklearn.model_selection import train_test_split

# Split data into X and y
y=df['churn'].copy()

# Define X

X = df.drop(columns=['churn'], axis=1)



#importing train_test_split
from sklearn.model_selection import train_test_split
# Split the data into a training and a test set and set stratify=y to help with imbalance data
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,
                                                 random_state=42)

df.head()

,tenure,monthlycharges,churn,gender_Male,seniorcitizen_Yes,partner_Yes,dependents_Yes,phoneservice_Yes,multiplelines_No phone service,multiplelines_Yes,...,totalcharges_995.35,totalcharges_996.45,totalcharges_996.85,totalcharges_996.95,totalcharges_997.65,totalcharges_997.75,totalcharges_998.1,totalcharges_999.45,totalcharges_999.8,totalcharges_999.9
0,1,29.85,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,34,56.95,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,53.85,1,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,45,42.30,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,2,70.70,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
### define our metrics function with plotting confusion matrix
import itertools
from sklearn.metrics import confusion_matrix 

### define function for plotting confusion matrix
def plot_confusion_matrix(y_true, y_preds):
    # Print confusion matrix
    cnf_matrix = confusion_matrix(y_true, y_preds)
    # Create the basic matrix
    plt.imshow(cnf_matrix,  cmap=plt.cm.Blues)
    # Add title and axis labels
    plt.title('Confusion Matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # Add appropriate axis scales
    class_names = set(y) # Get class labels to add to matrix
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=0)
    plt.yticks(tick_marks, class_names)
    # Add labels to each cell
    thresh = cnf_matrix.max() / 2. # Used for text coloring below
    # Here we iterate through the confusion matrix and append labels to our visualization
    for i, j in itertools.product(range(cnf_matrix.shape[0]), range(cnf_matrix.shape[1])):
            plt.text(j, i, cnf_matrix[i, j],
                     horizontalalignment='center',
                     color='white' if cnf_matrix[i, j] > thresh else 'black')
    # Add a legend
    plt.colorbar();
    plt.show();
def metrics(model_name, y_train, y_test, y_hat_train, y_hat_test):
    '''Print out the evaluation metrics for a given models predictions'''
    print(f'Model: {model_name}', )
    print('-'*60)
    plot_confusion_matrix(y_test,y_hat_test)
    print(f'test accuracy: {accuracy_score(y_test, y_hat_test)}')
    print(f'train accuracy: {accuracy_score(y_train, y_hat_train)}')
    print('-'*60)
    print('-'*60)
    print('Confusion Matrix:\n', pd.crosstab(y_test, y_hat_test, rownames=['Actual'], colnames=['Predicted'],margins = True))
    print('\ntest report:\n' + classification_report(y_test, y_hat_test))
    print('~'*60)
    print('\ntrain report:\n' + classification_report(y_train, y_hat_train))
    print('-'*60)

In [17]:
# # Build a  logistic regression base model using statsmodels

import statsmodels.api as sm
# define y
y=df['churn'].copy()

# Define X

X = df.drop(columns=['churn'], axis=1)


# Create intercept term required for sm.Logit, see documentation for more information
X = sm.add_constant(X)

# Fit model
logit_model = sm.Logit(y, X)

# Get results of the fit
result = logit_model.fit()
result.summary()

         Current function value: inf
         Iterations: 35


LinAlgError: Singular matrix

In [ ]:
#dropping  manually columns with p-value >0.05 that were not significant based on their higher p-values.

df.drop(['monthlycharges','gender_Male','partner_Yes','dependents_Yes','onlinesecurity_Yes','onlinebackup_Yes',
 'deviceprotection_Yes','techsupport_Yes','streamingtv_Yes','streamingmovies_Yes','paymentmethod_Credit card (automatic)',
 'paymentmethod_Mailed check'],axis=1, inplace=True)
display(df.columns)
X=df.drop(columns=['churn'], axis=1) ### redine X after dropping insignificant feature
X
df.info()

In [ ]:
#scale numeric features 
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
# Scale the train and test data
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
## logistic regression with sickit learn before smote
logreg = LogisticRegression()
base_log = logreg.fit(X_train, y_train)
base_log
#predictions
y_hat_train=base_log.predict(X_train)
y_hat_test = base_log.predict(X_test)

# model results

metrics(base_log, y_train, y_test, y_hat_train, y_hat_test)
# To get the  coffients of all the variables of logistic Regression
base_log_cof = pd.Series(base_log.coef_[0], index=X.columns.values)
print(base_log_cof)

base_log_cof.sort_values(inplace=True)
plt.figure(figsize=(15, 6))
plt.xticks(rotation=90)
features=plt.bar(base_log_cof.index,base_log_cof.values)